In [1]:
import numpy as np
import pandas as pd 
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import nltk
import string
import torch
from transformers import RobertaTokenizer, RobertaModel
from sklearn.manifold import TSNE
import umap
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.manifold import TSNE
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Open the zip file
with zipfile.ZipFile('data.zip') as z:
    # Open the JSON file
    with z.open('data/renttherunway_final_data.json') as f:
        df_rtr = pd.read_json(f, lines=True)

df_rtr.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10.0,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"


In [3]:
# Combine review_summary and review_text
df_rtr['combined_text'] = df_rtr['review_summary'] + " " + df_rtr['review_text']

In [4]:
# Encode Target Variable: Convert the 'fit' categories into numerical labels

custom_order = np.array(['small', 'fit', 'large'])

# Create LabelEncoder and set classes_
le = LabelEncoder()
le.classes_ = custom_order

# Transform target variable
df_rtr['fit_encoded'] = le.transform(df_rtr['fit'])

In [5]:
# Train-test split
X = df_rtr['combined_text']
y = df_rtr['fit_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available



In [7]:
# Tokenize and batch process
texts = df_rtr['combined_text'].tolist()
batch_size = 8  # Adjust based on memory availability
all_embeddings = []



In [8]:
# Process in batches
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors='pt')
    inputs = {key: val.to('cuda' if torch.cuda.is_available() else 'cpu') for key, val in inputs.items()}  # Move to GPU if available

    with torch.no_grad():
        outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Average token embeddings
        all_embeddings.append(batch_embeddings)



In [9]:
# Concatenate all embeddings into a single array
embeddings = np.vstack(all_embeddings)

In [10]:
###

In [11]:
# Load pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

In [12]:
# Tokenize text
inputs = tokenizer(df_rtr['combined_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

In [13]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased')

In [14]:
inputs = tokenizer(df_rtr['combined_text'].tolist(), 
                   padding=True, 
                   truncation=True, 
                   max_length=128, 
                   return_tensors='pt')

In [ ]:
# Ensure you have combined_text in your DataFrame
df_rtr['combined_text'] = df_rtr['review_summary'] + " " + df_rtr['review_text']

# Extract the text data
texts = df_rtr['combined_text'].tolist()  # Convert to a list for batch processing

# Batch Processing - loops through texts in chunks of size batch_size to process them without overwhelming memory.
batch_size = 8 # Start with a small batch size to avoid crashes
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
    
    with torch.no_grad():
        outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
        np.save(f'batch_{i}.npy', batch_embeddings)

# Load and concatenate embeddings
embeddings = np.concatenate([np.load(f'batch_{i}.npy') for i in range(0, len(texts), batch_size)])